In [12]:
import os
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set()
from config import DATA_FOLDER
from analysis import check_gies
import pandas as pd
import xarray as xr
import numpy as np

### Dataset to use - change this value to analyze a different data set

In [2]:
dataset = 'twenty'

### Load in parent probabilities and rates

In [3]:
dag_folders = check_gies.get_dag_folders(dataset)
true_dags = check_gies.get_true_dags(dag_folders)
print('loaded DAGs')
parent_probs_by_dag = check_gies.get_parent_probs_by_dag(dag_folders, 10)
print('Computed parent probabilities')
rates_by_dag = check_gies.get_rates_by_dag(parent_probs_by_dag, true_dags, 10)
print('Computed rates')

loaded DAGs
Loading parent probabilities for DAG 0
Loading parent probabilities for DAG 10
Loading parent probabilities for DAG 20
Loading parent probabilities for DAG 30
Loading parent probabilities for DAG 40
Computed parent probabilities
Loading parent probabilities for DAG 0
Loading parent probabilities for DAG 10
Loading parent probabilities for DAG 20
Loading parent probabilities for DAG 30
Loading parent probabilities for DAG 40
Computed rates


In [20]:
strategies = list(rates_by_dag[0].keys())
rate_names = list(rates_by_dag[0][strategies[0]].keys())
strategy2ix = {s: ix for ix, s in enumerate(strategies)}
rate2ix = {r: ix for ix, r in enumerate(rates)}

In [21]:
print(strategies)
print(rate_names)

['learn-parents,n=60,b=2,k=2', 'learn-parents,n=120,b=2,k=2', 'random,n=60,b=2,k=2', 'random,n=120,b=2,k=2', 'edge-prob,n=120,b=2,k=2', 'edge-prob,n=60,b=2,k=2']
['tpr', 'fpr', 'tnr', 'fnr', 'ppv', 'npv']


In [22]:
rate_array = np.zeros([len(strategies), len(rates), len(rates_by_dag)])
for d_ix, rates_by_strategy in enumerate(rates_by_dag):
    for strategy, rates in rates_by_strategy.items():
        s_ix = strategy2ix[strategy]
        for rate, val in rates.items():
            r_ix = rate2ix[rate]
            rate_array[s_ix, r_ix, d_ix] = val

In [23]:
ds = xr.DataArray(
    rate_array, 
    coords=[strategies, rate_names, range(len(rates_by_dag))],
    dims=['strategies', 'rates', 'dags'], 
)

In [38]:
avg_rates = ds.mean(dim='dags')
avg_tprs = rate_means.sel(rates='tpr')
avg_fprs = rate_means.sel(rates='fpr')

In [39]:
strategies

['learn-parents,n=60,b=2,k=2',
 'learn-parents,n=120,b=2,k=2',
 'random,n=60,b=2,k=2',
 'random,n=120,b=2,k=2',
 'edge-prob,n=120,b=2,k=2',
 'edge-prob,n=60,b=2,k=2']

In [40]:
avg_tprs

<xarray.DataArray (strategies: 6)>
array([0.407817, 0.473349, 0.430651, 0.484651, 0.506524, 0.423429])
Coordinates:
  * strategies  (strategies) <U27 'learn-parents,n=60,b=2,k=2' ...
    rates       <U3 'tpr'